<h1><center>~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~</center></h1>
<h1 style="font-family:verdana;"> <center> Data Science - Final Assignment </center> </h1>
<h1><center>~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~</center></h1>

<center>
<img src = "https://comicvine1.cbsistatic.com/uploads/original/11135/111350799/6404749-4102173126-144401.jpeg"></img>
</center>

<h1> Part I: <u>  <center>  Business & Data Understanding  </center> </u> </h1>

## Introduction
I present you my final assignment in the course "Data Science", studied at Ben Gurion University.

For my assignment I decided to explore and deepen my knowldege with **recommendation systems**, as I was very intrigued about this subject that we learned about in lesson 5, and even though we had a practical exercise back then, I felt like there's more that I want to try and experience for myself.

As a huge anime fan, I searched for a dataset that could bright my day and what could be a better collection of data other than over 85K user reviews of anime, scraped from the site https://myanimelist.net/, best site there is to all anime fans out there?!?!

## Problem - 

The problem we are dealing with today is actually a pretty well known problem - every business wants to recommend their clients products they would like to buy, or in our case - watch!

By doing so, people will stay longer in their sites, and the buisness will earn more money. 

It's a win-win scenario where everybody's happy.

Of Course, it's no secret that there are **countless** of ways to solve this kind of problem, but I think that by trying tools we didn't actually touched in the course, and by adding my own small improvments, this assignment would be uniqe enough in its own way.
(I certaintly hope so!)

In lesson 5's exercise we saw how calculating simlarity score between users can help us predict a recommendation, but in the anime world - new animes are coming out regulary, and we won't have enough information to make a recommendation by using the same tech.

We should try new methods that we didn't try back then and see how they compare to each other while also using a smart filtering of the data for refining the results! 

To begin this exploratory analysis, first I import libraries and define functions for plotting the data using `matplotlib`.

In [ ]:
!pip install --upgrade pip

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


We currently have 3 files, lets check their names:

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


### Let's check the file: "MAL Anime Reviews 85k.csv"

This is the main file containing the most important data for this project.
I'm also opening a file that gives extra details for the shows



In [ ]:
nRowsRead = None # specify 'None' if want to read whole file
# MAL Anime Top 10000 Details.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
anime_df = pd.read_csv('/kaggle/input/MAL Anime Reviews 85k.csv', delimiter=',', nrows = nRowsRead)
anime_details_df = pd.read_csv('/kaggle/input/MAL Anime Top 10000 Details.csv', delimiter=',', nrows = nRowsRead)
anime_df.dataframeName = 'MAL Anime Reviews 85k.csv'
anime_details_df.dataframeName = 'MAL Anime Top 10000 Details.csv'
nRow, nCol = anime_df.shape
print(f'There are {nRow} rows and {nCol} columns')

Let's take a quick look at what the data looks like:
we can see that we have data about users and their review for anime shows based on different criteria, its seems like a good start.
let's take a look on the user counts - users that reviewd more than once are important to create a decent model.

In [ ]:
anime_df.head(5)

In [ ]:
user_counts = anime_df['Username'].value_counts()
user_counts = user_counts.value_counts()
user_counts[0:15].plot(kind = 'area', title = 'number of users that rated X animes')

We can see that most of the users in our data only reviewd 1 anime, and the numbers are getting smaller and smaller drastically.

Considered that there are thousands of different animes in our data, it makes a really poor data to make a personalized recommendation, so we are going to need thinking about a 'cold start' model that will help us give recommmendation for users. 

lets look at our anime sparse data:

In [ ]:
anime_counts = anime_df['Anime Title'].value_counts()
anime_counts = anime_counts.value_counts()
anime_counts
anime_counts[0:15].plot(kind = 'pie', title = 'number of anime that rated X times')

I'll filter out users with less than 10 reviews for animes:

In [ ]:
counts = anime_df['Username'].value_counts()
anime_df = anime_df[anime_df['Username'].isin(counts[counts > 10].index)]

<h1> Part II: <u>  <center>  Data Preparation  </center> </u> </h1>

Removing columns and duplications:

These columns are pretty weird. I don't think we need them in this project

In [ ]:
anime_df = anime_df.drop_duplicates()
anime_df = anime_df.drop(['Anime URL','Review Date','Review'],axis = 1)
anime_df

We can see that we have a pretty cool column named "Episodes Watched".

It tells us how many episodes of the anime the user watched before submitting the review.

I don't know much about you, but I think a decent review should come from someone who watched as many episodes as possible from the show.
(There are many great animes that starts bad but improve a lot with time)

This data could be great for us, but we can't really use it in its current state, we need to change it.

In [ ]:
print(anime_df['Episodes Watched'].dtypes)
print(anime_df['Episodes Watched'][0])

print(anime_df.info())

Lets build a function that will transform this column to numerical one. 
instead of a line stating how many episodes the user watched I want to know about it in percents.

In [ ]:
def show_completion_percent(line):
    array = [int(s) for s in line.split() if s.isdigit()]
    if len(array) == 2 and array[1] != 0:
        return array[0]/array[1]
    else:
        return -1
        
    
print("Test: " + str(show_completion_percent("64 of 64 episodes seen") * 100) + "%")
anime_df['Episodes Watched'] = [show_completion_percent(i) for i in anime_df['Episodes Watched']]

Lets see how many records we are going to lose after cleaning the users that didn't give a show a proper chance:

In [ ]:
anime_df.head()
count = 0
for i in anime_df['Episodes Watched'].values:
    if i <= 0.5:
        count += 1
print(count)

In [ ]:
anime_df = anime_df[anime_df['Episodes Watched'] > 0.5]
anime_df

Now I want to create a target column of rating which will be the mean value of the columns - Story, Animation, Sound, Character and enjoyment. 

That will be the new "Overall Rating" of a user to an anime.
I really think that the overall rating for a show supposed to be consisted of the other measures.

It doesn't make sense giving a show a score of 10/10 for story, animation, sound, character and enjoyment and then say that your **overall rating is 0**.

That would be super silly don't you agree with me?
I'm pretty sure that there are few records like that in this huge dataset and we don't want any of that misleading information in our data.

Also, I'm changing possible scores of 0 to be 0.1, because later we are going to use 0 as indication for that the user didn't rate an anime at all

In [ ]:
rating_average = anime_df.iloc[:,6:11].mean(axis=1)
rating_average = rating_average
for i in rating_average:
    if i == 0:
        i = 0.1
        
rating_average


In [ ]:
anime_df = anime_df.copy()
anime_df.loc[0:,'Average Overall Rating'] = rating_average
anime_df

Now we are going to rid of anyone who have a difference of |Average Overall Rating - Overall Rating| > 2

In [ ]:
anime_df = anime_df[abs(anime_df['Overall Rating'] - anime_df['Average Overall Rating']) <= 2]
anime_df.drop(['Overall Rating'], axis = 1)
anime_df

Now, I want to use the "Wisdom of the crowd" to create features for anime shows. 

In [ ]:
anime_features_df = anime_df.groupby(['Anime Title','Anime Rank'])[['Story Rating','Animation Rating','Sound Rating','Character Rating','Enjoyment Rating']].mean()
anime_features_df['Mean'] = anime_features_df.mean(axis = 1)
anime_features_df

Now, lets leave the features table aside for a moment and create a matrix table of Users X Anime with the values of their respective Overall score.

In [ ]:
anime_sparse_table = anime_df.pivot_table(index='Username',columns='Anime Title',values='Average Overall Rating').fillna(0)
anime_sparse_table

Now we can see that this table is very **very** sparsed!
Simple glimpse into that table right now shows no values at all, thats because there are so many animes(=items) and most of the people didn't even watch them. That's an example for a reality challenge.

I'll split the data into train & test,
our test will be 15% of the table 

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(anime_sparse_table, test_size = 0.15)


In [ ]:
test

<h1> Part III: <u>  <center> Modeling  </center> </u> </h1>

# **Collaborative Filtering - Item Based**

In item based model we are trying to recommend an item to a user based on **similar items** instead of user - simlarity like we have done in lesson 5.

Our data will usually contain many more users than items, so computing a recommendation that way should be faster and somewhat more powerful.

First I am calculating the pearson correlation between the columns in the matrix (the animes)
that way we can check most similiar animes to a specific anime.

In [ ]:
corr_between_anime = train.corr(method='pearson')

In [ ]:
corr_between_anime

Lets check what are the most similiar animes to 'Dragon Ball':

<center>
<img src = "https://wallpapercave.com/wp/wp6392800.jpg"></img>
</center>

In [ ]:
corr_between_anime['Dragon Ball Z'].sort_values(ascending = False).head()

Now it's time to create the prediction function:

In [ ]:
def top_k_similiar_anime(anime, k = 5):
    temp = corr_between_anime[anime].sort_values(ascending = False)
    top_similiar = []
    for i in range(1,6):
        top_similiar.append(temp.index[i])
    return top_similiar

def get_anime_user_rated(user,df):
    anime_user_rated = []
    count = 0
    for i in df.loc[user]:
        if i > 0:
            anime_user_rated.append(df.loc[user].index[count])
        count += 1
    return anime_user_rated

def is_nan(x):
    return (x != x)
        
def predict_score(user, anime, df, k = 6):
    anime_user_rated = get_anime_user_rated(user,df)
    prediction = None
    if len(anime_user_rated) >= 6:
        corr_table = corr_between_anime[anime].loc[anime_user_rated].sort_values(ascending = False)
        mone = 0
        mehane = 0
        for i in range(1,k):
            mone += corr_table[i] * df[corr_table.index[i]].loc[user]
            mehane += corr_table[i]
    
        prediction = mone / mehane
    
    if prediction is None or is_nan(prediction) or prediction < 0:
        prediction = anime_features_df['Mean'][anime].values[0]    
    return prediction
        

My prediction function makes a prediction based on the ratings the user gave to top K most similiar animes = (Neareast Neighbors),
If there isn't enough information I'm using the global average score of the anime as the prediction - and that would be beneficial for the 'cold start' problem.

Now, in order to save time I will predict the score for 100 users in the train data, only on animes they did review.

In [ ]:
hundred_users_with_over_ten_reviews = []
count = 0
for user in train.index:
    if count != 100:
        animes_user_rated = get_anime_user_rated(user,train)
        if len(animes_user_rated) >= 10:
            hundred_users_with_over_ten_reviews.append(user)
            count += 1

hundred_users_with_over_ten_reviews

In [ ]:
real_scores_train = []
predicted_scores_train = []
for anime in train.columns:
    for user in hundred_users_with_over_ten_reviews:
        if train[anime][user] > 0:
            real_scores_train.append(train[anime][user])
            pred = predict_score(user,anime,train)
            predicted_scores_train.append(pred)
            print('Username: ' + str(user) + ' Rated Anime: ' + str(anime) + ' score: ' + str(train[anime][user]))
            print('The Predicted Score however is: ' + str(pred))

Time to check the error margins:

In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE: = ' + str(mean_squared_error(real_scores_train, predicted_scores_train)))
print('RMSE: = ' + str(mean_squared_error(real_scores_train, predicted_scores_train,squared = False)))

Now I'll do the same with the test: (While using the similiarity score of the train)

In [ ]:
users_test = []
for user in test.index:
        animes_user_rated = get_anime_user_rated(user,test)
        if len(animes_user_rated) >= 10:
            users_test.append(user)

users_test

In [ ]:
real_scores_test = []
predicted_scores_test = []
for anime in test.columns:
    for user in users_test:
        if test[anime][user] > 0:
            real_scores_test.append(test[anime][user])
            pred = predict_score(user,anime,test)
            predicted_scores_test.append(pred)
            print('Username: ' + str(user) + ' Rated Anime: ' + str(anime) + ' score: ' + str(test[anime][user]))
            print('The Predicted Score however is: ' + str(pred))

In [ ]:
print('MSE: = ' + str(mean_squared_error(real_scores_test, predicted_scores_test)))
print('RMSE: = ' + str(mean_squared_error(real_scores_test, predicted_scores_test,squared = False)))

<h1>  CTR: DeepFM - Model </h1>

Time to merge our anime features table from above with a table that contains more features about each anime title in this list.

In [ ]:
anime_details_df.info()
fusion_table = pd.merge(anime_details_df,anime_features_df,left_on='Ranking',right_on='Anime Rank')

In [ ]:
fusion_table.info()
fusion_table = fusion_table.iloc[:,[0,12,14,23,24,25,26,27]]
fusion_table

In [ ]:
anime_df = anime_df.drop(['Animation Rating','Sound Rating','Character Rating','Enjoyment Rating', 'Story Rating'],axis = 1)
fusion_table = pd.merge(fusion_table,anime_df,on='Anime Title')
fusion_table

In [ ]:
fusion_table.columns

In [ ]:
new_columns_name = ['Anime_Title', 'Studios', 'Genres', 'Story_Rating', 'Animation_Rating',
       'Sound_Rating', 'Character_Rating', 'Enjoyment_Rating', 'Anime_Rank',
       'Username', 'Episodes_Watched', 'Review_Likes', 'Overall_Rating',
       'Average_Overall_Rating']
fusion_table.columns = new_columns_name
fusion_table

In [ ]:
fusion_table["Genres"]
dummies = fusion_table['Genres'].str.get_dummies(sep=',')
#dummies
final_fusion_table = pd.concat([fusion_table,dummies],axis=1)
final_fusion_table = final_fusion_table.rename(columns={"Martial Arts": "Martial_Arts", "Slice of Life": "Slice_of_Life", "Super Power": "Super_Power"})
columns = final_fusion_table.columns

#columns.delete(['Average_Overall_Rating','Overall_Rating'])
columns_array = []
for i in columns:
    if i[0] != ' ':
        columns_array.append(i)
columns_array.remove('Average_Overall_Rating')
columns_array.remove('Overall_Rating')
columns_array.remove('Studios')
columns_array.remove('Genres')
columns_array
genres = columns_array[10:]
columns_array

#fusion_table = pd.get_dummies(fusion_table, columns = ['Studios'])

In [ ]:
!pip install git+https://github.com/shenweichen/DeepCTR

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat,get_feature_names

data = final_fusion_table
sparse_features = columns_array
target = ['Average_Overall_Rating']

1. Label Encoding for sparse features,and do simple Transformation for dense features

In [ ]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

2. count #unique features for each sparse field

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(),embedding_dim=4)
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

3. splitting the data to train and test

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

* 4. Define Model,train, and predict
Time to see the DeepFM model in action:

In [ ]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)
pred_ans

<h1> Part IV: <u>  <center> Evaluation [MSE]  </center> </u> </h1>

we can use the deepFM model MSE calculator to evaluate the Model.

In [ ]:
print("test MSE", round(mean_squared_error(test[target].values, pred_ans), 4))

## Conclusion

Today, we looked into 2 more ways to build a recommender system.
we have seen that real life data can be very poor and challenging to work with and that's why we need models that can hybrid between static info of animes and the personalized data of the users.

In terms of evaluation it seems like both models did pretty good job with predicting user scores for anime,
it seems like the DeepFM has the upperhand however, it used much more data in the calculation combining factorization machine (the ratings of the users and their seprated features).